# Task:
1. Simulate player gameplay 
2. Estimate their average spend
3. Determine fixed rewards for each completed level. The reward structure should be designed in such a way that the expected value of player rewards equals 1.3 times their
average spend (including any additional deposits).

### 1. Simulation of Gameplay
The player progresses through the levels by playing slot games. Upon reaching specific multipliers (e.g. x25, x50, etc.) in a limited time, the player advances to the next level. 

##### i. Player progression:
- Player enters a challenge at level 1 with a non zero bankroll (approx. 50 EUR)
- Players must hit or exceed the target multiplier in the given time.
- If they reached the next level the unused time is added to the next level.
- Players can add additional funds up to 4 times.

##### ii. End of the game:
- Players do not reach or surpass the target multiplaier in the given time
- Players reach the max multiplaier
- Players run out of money

### 2. Exploration of Variables

How results vary based on: (given a slot machine)
- initial bankroll size
- additional deposit (max 4)
- bet size: 1, 2, 5, 10, 20, 50 €

### 3. Reward Design

- Define a fixed monetary reward for each level.
- Ensure the prize structure logically scales with level difficulty and encourages
progression.
- The target is for the expected value of the rewards to equal 1.3× the player’s average
spend, including all deposits. Example: A player deposits a total €100 of their funds
into playing; the rewards should total €130.


### 4. Key metrics:

- player distribution across the levels
- avg. spend per level for all levels
- avg. time spend per level (potential indicator for player engagement)
- avg. reward per level



### 5. Assumptions:
- Spin time: 
    - 2 s (autospin)
    - 5 s (player input) 
- Bet size: 1, 2, 5, 10, 20, 50 € (just mimicking the currency options)

### Used libraries and given data
Numpy, matplotlib, pandas, ipytest

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import ipytest
ipytest.autoconfig()         # picks up `pytest.ini` in cwd, if any



In [ ]:
slot_machine_levels = {
    1: {
        "target_multiplier": 25,
        "time_limit": "30 min",
        "time_seconds": 30 * 60
    },
    2: {
        "target_multiplier": 50,
        "time_limit": "1 hour",
        "time_seconds": 1 * 60 * 60
    },
    3: {
        "target_multiplier": 100,
        "time_limit": "2 hours",
        "time_seconds": 2 * 60 * 60
    },
    4: {
        "target_multiplier": 200,
        "time_limit": "3 hours",
        "time_seconds": 3 * 60 * 60
    },
    5: {
        "target_multiplier": 500,
        "time_limit": "4 hours",
        "time_seconds": 4 * 60 * 60
    },
    6: {
        "target_multiplier": 800,
        "time_limit": "6 hours",
        "time_seconds": 6 * 60 * 60
    },
    7: {
        "target_multiplier": 1500,
        "time_limit": "8 hours",
        "time_seconds": 8 * 60 * 60
    },
    8: {
        "target_multiplier": 2500,
        "time_limit": "12 hours",
        "time_seconds": 12 * 60 * 60
    },
    9: {
        "target_multiplier": 5000,
        "time_limit": "24 hours",
        "time_seconds": 24 * 60 * 60
    },
    10: {
        "target_multiplier": 10000,
        "time_limit":"48 hours",
        "time_seconds":48 * 60 * 60 
    }
}

print(slot_machine_levels)

slot1 = np.load('data/multipliers_slot_1.npy')
slot2 = np.load('data/multipliers_slot_2.npy')
print(type(slot1), slot1.shape)

{1: {'target_multiplier': 25, 'time_limit': '30 min', 'time_seconds': 1800}, 2: {'target_multiplier': 50, 'time_limit': '1 hour', 'time_seconds': 3600}, 3: {'target_multiplier': 100, 'time_limit': '2 hours', 'time_seconds': 7200}, 4: {'target_multiplier': 200, 'time_limit': '3 hours', 'time_seconds': 10800}, 5: {'target_multiplier': 500, 'time_limit': '4 hours', 'time_seconds': 14400}, 6: {'target_multiplier': 800, 'time_limit': '6 hours', 'time_seconds': 21600}, 7: {'target_multiplier': 1500, 'time_limit': '8 hours', 'time_seconds': 28800}, 8: {'target_multiplier': 2500, 'time_limit': '12 hours', 'time_seconds': 43200}, 9: {'target_multiplier': 5000, 'time_limit': '24 hours', 'time_seconds': 86400}}
<class 'numpy.ndarray'> (10000000,)


### Slot statistics
Just a quick look at the slots statitics.

In [3]:
slot1_series = pd.Series(slot1, name="Slot 1")
slot2_series = pd.Series(slot2, name="Slot 2")

df_slot_stats = pd.concat([slot1_series, slot2_series], axis=1)

print(df_slot_stats.describe())


             Slot 1        Slot 2
count  1.000000e+07  2.000000e+07
mean   9.509598e-01  9.670817e-01
std    6.934455e+00  1.232296e+01
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    0.000000e+00  0.000000e+00
75%    1.666000e-01  0.000000e+00
max    7.000000e+02  1.470825e+04


### Spinning
Simulating the slot spin rezult with uniformly choosing a slot outcome.

In [ ]:

def spin(slot):
    random_value = np.random.choice(slot)
    print("pull", random_value)
    return random_value


In [207]:

def check_multi(level, current_multiplier, used_spins, spin_time):
    """
    Returns: next_level, remaining_time
      next_level (int) if current_multiplier >= target and elapsed_seconds <= time_limit
      1            if the player failed (reset to level 1)
    """
    info = slot_machine_levels.get(level)
    time_limit = info["time_seconds"]
    target = info["target_multiplier"]
    remaining_time = slot_machine_levels[level]["time_seconds"]/spin_time - used_spins


    if not info:
        # invalid level: reset
        return 1, 0.0

    if (current_multiplier >= info["target_multiplier"]
        and remaining_time >= 0):
        next_level = level + 1
        # remaining_time = slot_machine_levels[level]["time_seconds"]/spin_time - used_spins
        return next_level, remaining_time
    else:
        # failure → reset
        return 1, 0.0
    
print(check_multi(1, 30, 10*60, 2))
def test_check_multi():
    # player next lvl reached multi = 30 > target = 25 in 10 min, 20 min remains
    assert(check_multi(1, 30, 10*60, 1)) == (2, 1200)
    # player loses per multiplier
    assert(check_multi(1, 10, 600, 1))== (1, 0.0)
    # player loses per time
    assert(check_multi(1, 30, 31*60, 1))== (1, 0.0)
    assert(check_multi(1, 30, 5*60, 2)) == (2, 600)
    assert False, "This test is designed to always fail"



(2, 300.0)


### User Cash Flow Oversight
assumption u add money only when u cant bet anymore

In [200]:
def cash_flow(bet_size: float, balance: float, deposit: float, added_fund: int):
    total_spend = 0

    if balance >=0.0 and added_fund<4:
        if bet_size>=balance:
            balance += deposit
            added_fund += 1
            balance -= bet_size
        total_spend = balance + bet_size

    
    if balance < bet_size and added_fund==4:
        bet_size = balance
        total_spend += bet_size
        print("changing bet size")
    
    return total_spend, balance, added_fund

print(cash_flow(10.0, 0.0, 0.0, 4))
def test_cash_flow():
    # positive balance
    assert(cash_flow(0, 10.0, 0, 0) == (10.0, 10.0, 0))
    # no balance
    assert(cash_flow(20.0, 0, 0, 4) == (0.0, 0.0, 4))
    # add deposit
    assert(cash_flow(11.0, 10.0, 20.0, 1))==(30.0, 19.0, 2)
    # last deposit
    assert(cash_flow(15, 10, 5, 3))==(15, 0, 4)
    # a test that fails
    #assert False, "This test is designed to always fail"

changing bet size
(0.0, 0.0, 4)


assumption
- players have a max of 48h to paticipate to the promo challange
- players want to maximize their time per spins --> using all the 48h for betting

In [261]:
max_time =20   #48*60*60 # s
auto_spin = 2 # s
human_avg_spin = 5 # s
start_lvl = 1

def players_gameplay(spin_time):
    current_multiplier = 0
    start_lvl = 1
    for i in range(int(max_time/spin_time)):
        
        # current_multiplier+=spin(slot1)
        a = check_multi(i+1, 0, 0, spin_time)
        print(a)
    return 0

players_gameplay(auto_spin)

(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)
(1, 0.0)


TypeError: 'NoneType' object is not subscriptable

In [202]:
# running tests
ipytest.run()

F.                                                                                           [100%]
============================================= FAILURES =============================================
_________________________________________ test_check_multi _________________________________________

    def test_check_multi():
        # player next lvl reached multi = 30 > target = 25 in 10 min, 20 min remains
        assert(check_multi(1, 30, 10*60, 1)) == (2, 1200)
        # player loses per multiplier
        assert(check_multi(1, 10, 600, 1))== (1, 0.0)
        # player loses per time
        assert(check_multi(1, 30, 31*60, 1))== (1, 0.0)
        assert(check_multi(1, 30, 5*60, 2)) == (2, 600)
>       assert False, "This test is designed to always fail"
E       AssertionError: This test is designed to always fail
E       assert False

/tmp/ipykernel_6331/1590133725.py:35: AssertionError
--------------------------------------- Captured stdout call --------------------------------

<ExitCode.TESTS_FAILED: 1>